In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/godaddy-microbusiness-density-forecasting/sample_submission.csv
/kaggle/input/godaddy-microbusiness-density-forecasting/census_starter.csv
/kaggle/input/godaddy-microbusiness-density-forecasting/train.csv
/kaggle/input/godaddy-microbusiness-density-forecasting/test.csv


In [2]:
data=pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/train.csv')
df=data.copy()

In [3]:
df.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243


In [4]:
df.describe(include='all')

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
count,122265,122265.000000,122265,122265,122265,122265.000000,1.222650e+05
unique,122265,NaN,1871,51,39,NaN,NaN
top,1001_2019-08-01,NaN,Washington County,Texas,2019-08-01,NaN,NaN
freq,1,NaN,1170,9906,3135,NaN,NaN
mean,NaN,30376.037640,NaN,NaN,NaN,3.817671,6.442858e+03
std,NaN,15143.508721,NaN,NaN,NaN,4.991087,3.304001e+04
min,NaN,1001.000000,NaN,NaN,NaN,0.000000,0.000000e+00
25%,NaN,18177.000000,NaN,NaN,NaN,1.639344,1.450000e+02
50%,NaN,29173.000000,NaN,NaN,NaN,2.586543,4.880000e+02
75%,NaN,45077.000000,NaN,NaN,NaN,4.519231,2.124000e+03


In [5]:
import pandas as pd

def fill_missing_values(df):
    for col in df.columns:
        # Calculate the percentage of missing values
        pct_missing = df[col].isnull().mean()
        
        # If more than 50% of the values are missing, drop the column
        if pct_missing > 0.5:
            df.drop(col, axis=1, inplace=True)
            continue
        
        # Fill missing values according to the data type of the column
        if df[col].dtype == 'object':
            df[col].fillna(df[col].mode()[0], inplace=True)
        elif df[col].dtype == 'float':
            df[col].fillna(df[col].mean(), inplace=True)
        elif df[col].dtype == 'datetime64[ns]':
            df[col].fillna(df[col].mode()[0], inplace=True)
        else:
            df[col].fillna(df[col].median(), inplace=True)
            
    return df


In [6]:
df = fill_missing_values(df)


In [7]:
df.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243


In [8]:
import pandas as pd

# Check for missing values
df.isnull().sum()

# Handle missing values
df = df.dropna()  # drop rows with missing values

# Ensure that dates are in the correct format
df['first_day_of_month'] = pd.to_datetime(df['first_day_of_month'])

# Resolve inconsistencies or errors in the data
df['county'] = df['county'].str.replace(' County', '')  # remove " County" from county names


In [9]:
df

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga,Alabama,2019-12-01,2.993233,1243
...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston,Wyoming,2022-06-01,1.803249,101
122261,56045_2022-07-01,56045,Weston,Wyoming,2022-07-01,1.803249,101
122262,56045_2022-08-01,56045,Weston,Wyoming,2022-08-01,1.785395,100
122263,56045_2022-09-01,56045,Weston,Wyoming,2022-09-01,1.785395,100


In [10]:
import pandas as pd
# Select only the numeric features
df_numeric = df.select_dtypes(include='number')

# Select only the categorical features
df_categorical = df.select_dtypes(include='object')

# Select a subset of features by name
selected_features = ['cfips', 'county', 'state', 'microbusiness_density']
df_subset = df[selected_features]

# Drop a subset of features by name
dropped_features = ['row_id', 'first_day_of_month']
df_subset = df.drop(columns=dropped_features)

def preprocess_data(df):
    # Replace missing values with the mean of the column
    df = df.fillna(df.mean(numeric_only=True))
    
    # Standardize the values
    df = (df - df.mean(numeric_only=True)) / df.std(numeric_only=True)
    
    # Normalize the values
    df = (df - df.min()) / (df.max() - df.min())
    
    return df

# Preprocess the data
df_preprocessed = preprocess_data(df_numeric)

def preprocess_categorical_columns(df):
  # Encode the 'county' and 'state' columns
  le = LabelEncoder()
  df.loc[:, 'county'] = le.fit_transform(df['county'])
  df.loc[:, 'state'] = le.fit_transform(df['state'])

  return df
preprocessed_df1 = preprocess_categorical_columns(df_categorical)

dfpreprocessed_df1= preprocessed_df1.drop('row_id', axis=1)


def concat_dataframes(df1, df2):
  # Concatenate the two DataFrames along the rows
  df = pd.concat([df1, df2], axis=1)

  return df
df=concat_dataframes(dfpreprocessed_df1,df_preprocessed)

NameError: name 'LabelEncoder' is not defined

In [ ]:
df.describe()

In [ ]:
import matplotlib.pyplot as plt

def plot_column_distributions(df):
  # Plot the histogram for each numeric column
  df.hist(figsize=(10, 10))

  # Show the plot
  plt.show()


In [ ]:
plot_column_distributions(df)
